In [1]:
from difflogic import LogicLayer, GroupSum
import torch
from chop import MaseGraph

/rds/general/user/jj21/home/miniforge3/envs/adls-project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os
# import sys

# "Add dot (pydot) to path if needed:"

# new_path = os.path.expanduser("~/miniforge3/envs/adls-project/bin/")
# if new_path not in sys.path:
#     sys.path.append(new_path)

# # Add to environment PATH as well
# os.environ["PATH"] = new_path + os.pathsep + os.environ["PATH"]

In [3]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    LogicLayer(400, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    LogicLayer(8_000, 8_000),
    GroupSum(k=10, tau=20)
)

In [4]:
mg = MaseGraph(model)

mg.draw("DLG.svg")

In [5]:
def eval(model, loader, mode):
    orig_mode = model.training
    with torch.no_grad():
        model.train(mode=mode)
        res = np.mean(
            [
                (model(x.to('cpu').round()).argmax(-1) == y.to('cpu')).to(torch.float32).mean().item()
                for x, y in loader
            ]
        )
        model.train(mode=orig_mode)
    return res.item()

In [6]:
import mnist_dataset
import numpy as np

train_set = mnist_dataset.MNIST('./data-mnist', train=True, download=True, remove_border=True)
test_set = mnist_dataset.MNIST('./data-mnist', train=False, remove_border=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, pin_memory=True, drop_last=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, pin_memory=True, drop_last=True)

state_dict = torch.load("best_model_full.pth", map_location=torch.device('cpu'))  # Load the dictionary
# state_dict = torch.load("best_model.pth", map_location=torch.device('cpu'))  # Load the dictionary
mg.model.load_state_dict(state_dict)  # Apply the weights

mg.model.eval()

print("Accuracy: ", eval(mg.model, test_loader, mode=False))

100%|██████████| 9.91M/9.91M [00:00<00:00, 14.5MB/s]


100%|██████████| 28.9k/28.9k [00:00<00:00, 379kB/s]


100%|██████████| 1.65M/1.65M [00:00<00:00, 3.54MB/s]


100%|██████████| 4.54k/4.54k [00:00<00:00, 2.99MB/s]



Accuracy:  0.9751000040769577
